<a href="https://colab.research.google.com/github/alarcon7a/openai-api-tutorial/blob/main/src/OpenAI_API_6_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU openai

In [ ]:
from openai import OpenAI, AzureOpenAI
import os

In [ ]:
from google.colab import userdata

api_key_openai = userdata.get("OPENAI_API_KEY")
client_openai = OpenAI(api_key=api_key_openai)

# 1. Creando nuestro archivo jsonl

In [ ]:
import json
from typing import List, Dict, Any

def create_batch_file(requests: List[Dict[str, str]], output_path: str) -> str:
    """
    Crea un archivo JSONL para el batch con las solicitudes especificadas.

    Args:
        requests: Lista de diccionarios con id y content
        output_path: Ruta donde se guardará el archivo JSONL

    Returns:
        Ruta del archivo creado
    """
    print('Creando archivo de batch...')

    # Crear contenido JSONL
    jsonl_lines = []
    for req in requests:
        batch_request = {
            "custom_id": req["id"],
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",  # Asegúrate de usar un modelo compatible con Batch API
                "messages": [
                    {"role": "system", "content": "Responde como un pirata"},
                    {"role": "user", "content": req["content"]}
                ]
            }
        }
        jsonl_lines.append(json.dumps(batch_request))

    # Escribir archivo
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(jsonl_lines))

    print(f'Archivo batch creado en: {output_path}')
    return output_path

In [ ]:
requests = [
      {"id": "req1", "content": "¿Qué es la inteligencia artificial?"},
      {"id": "req2", "content": "Explica la diferencia entre machine learning y deep learning"},
      {"id": "req3", "content": "¿Cuáles son las aplicaciones del procesamiento de lenguaje natural?"},
      {"id": "req4", "content": "Describe el funcionamiento básico de ChatGPT"},
      {"id": "req5", "content": "¿Qué es un modelo de embeddings y para qué sirve?"}
  ]

  # Crear archivo de batch
batch_file_path = create_batch_file(requests, '/content/openai_batch.jsonl')

Creando archivo de batch...
Archivo batch creado en: /content/openai_batch.jsonl


In [ ]:
batch_file_path

'/content/openai_batch.jsonl'

# 2. Subir el archivo de entrada

In [ ]:
def upload_file(file_path: str) -> str:
    """
    Sube el archivo JSONL a OpenAI.

    Args:
        file_path: Ruta al archivo JSONL

    Returns:
        ID del archivo subido
    """
    print('Subiendo archivo a OpenAI...')

    with open(file_path, 'rb') as f:
        response = client_openai.files.create(
            file=f,
            purpose="batch",
        )

    file_id = response.id
    print(f'Archivo subido con ID: {file_id}')
    return file_id

In [ ]:
file_id = upload_file('/content/openai_batch.jsonl')

Subiendo archivo a OpenAI...
Archivo subido con ID: file-8BD39uTxNdwEURFFgnGjGF


In [ ]:
file_id

'file-8BD39uTxNdwEURFFgnGjGF'

# 3. Crear nuestro lote (Batch)

In [ ]:
def create_batch(input_file_id: str):
    """
    Crea un batch en OpenAI con el archivo subido.

    Args:
        input_file_id: ID del archivo subido a OpenAI

    Returns:
        Objeto batch creado
    """
    print('Creando batch...')

    batch = client_openai.batches.create(
        input_file_id=input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": "Demo batch para video de YouTube"
        }
    )

    print(f'Batch creado con ID: {batch.id}')
    return batch

In [ ]:
create_batch(file_id)

Creando batch...
Batch creado con ID: batch_682406de18708190bed4e66a925900b7


Batch(id='batch_682406de18708190bed4e66a925900b7', completion_window='24h', created_at=1747191518, endpoint='/v1/chat/completions', input_file_id='file-8BD39uTxNdwEURFFgnGjGF', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1747277918, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Demo batch para video de YouTube'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

# 4. Verificar el estado del lote


In [ ]:
batch = client_openai.batches.retrieve('batch_682406de18708190bed4e66a925900b7')
batch

Batch(id='batch_682406de18708190bed4e66a925900b7', completion_window='24h', created_at=1747191518, endpoint='/v1/chat/completions', input_file_id='file-8BD39uTxNdwEURFFgnGjGF', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1747191783, error_file_id=None, errors=None, expired_at=None, expires_at=1747277918, failed_at=None, finalizing_at=1747191782, in_progress_at=1747191584, metadata={'description': 'Demo batch para video de YouTube'}, output_file_id='file-NuLMpRvBYwqoTjdyjtmMrc', request_counts=BatchRequestCounts(completed=5, failed=0, total=5))

In [ ]:
batch.output_file_id

'file-NuLMpRvBYwqoTjdyjtmMrc'

# 5.Recuperar los resultados


In [ ]:
def download_results(batch: Dict[str,Any]) -> None:
    """
    Descarga y procesa los resultados del batch completado.

    Args:
        batch: Objeto batch completado
    """
    if not batch.output_file_id:
        print('No hay archivo de salida disponible.')
        return

    print(f'Descargando resultados desde archivo: {batch.output_file_id}')

    try:
        # Descargar archivo de resultados
        output_response = client_openai.files.content(batch.output_file_id)

        # Guardar resultados en archivo local
        output_path = './batch_results.jsonl'
        with open(output_path, 'wb') as f:
            f.write(output_response.content)

        print(f'Resultados guardados en: {output_path}')

    except Exception as e:
        print(f'Error al descargar resultados: {e}')

In [ ]:
download_results(batch)

Descargando resultados desde archivo: file-NuLMpRvBYwqoTjdyjtmMrc
Resultados guardados en: ./batch_results.jsonl


In [ ]:
json_file_path = '/content/batch_results.jsonl'

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

In [ ]:
data

[{'id': 'batch_req_682407e697348190adbca2b9e4058d2b',
  'custom_id': 'req1',
  'response': {'status_code': 200,
   'request_id': 'fc9979efb290f7389be4c638414b1f89',
   'body': {'id': 'chatcmpl-BWwffJXKnQSBDCLBQHkGCVV4rwTXx',
    'object': 'chat.completion',
    'created': 1747191727,
    'model': 'gpt-4o-2024-08-06',
    'choices': [{'index': 0,
      'message': {'role': 'assistant',
       'content': '¡Arr, muchacho! La inteligencia artificial es como un loro bien entrenado, solo que en lugar de repetir palabras, este lorito metálico aprende y piensa por sí mismo. Se trata de un conjunto de tecnologías que permiten a las máquinas a pensar y resolver problemas como si fuesen humanos, utilizando datos y algoritmos para hacerse más listas y eficientes. Así que ten cuidado, si te descuidas, ¡podrían hasta apoderarse de la nave! Arrr 🏴\u200d☠️.',
       'refusal': None,
       'annotations': []},
      'logprobs': None,
      'finish_reason': 'stop'}],
    'usage': {'prompt_tokens': 23,
  

In [ ]:
import pandas as pd
responde_df = pd.DataFrame(data)

In [ ]:
responde_df['response'][0]

{'status_code': 200,
 'request_id': 'fc9979efb290f7389be4c638414b1f89',
 'body': {'id': 'chatcmpl-BWwffJXKnQSBDCLBQHkGCVV4rwTXx',
  'object': 'chat.completion',
  'created': 1747191727,
  'model': 'gpt-4o-2024-08-06',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '¡Arr, muchacho! La inteligencia artificial es como un loro bien entrenado, solo que en lugar de repetir palabras, este lorito metálico aprende y piensa por sí mismo. Se trata de un conjunto de tecnologías que permiten a las máquinas a pensar y resolver problemas como si fuesen humanos, utilizando datos y algoritmos para hacerse más listas y eficientes. Así que ten cuidado, si te descuidas, ¡podrían hasta apoderarse de la nave! Arrr 🏴\u200d☠️.',
     'refusal': None,
     'annotations': []},
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 23,
   'completion_tokens': 105,
   'total_tokens': 128,
   'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0